In [89]:
months = [
'Anytime',
'January',
'February',
'March',
'April',
'May',
'June',
'July',
'August',
'September',
'October',
'November',
'December'
]
intervals = [
    'Anytime',
    'Early',
    'Mid',
    'Late'
]
others = [
    'Female Only',
    'Male Only',
    'Pets',
    'Parking',
    'LGBTQ',
    'Private',
    '420',
]

In [85]:
# assumption: 
# 1. move in Month of a house cannot be anytime
# 2. early <= late is valid
def compareMonthV1(early,late,curr):
    interval_upper, month_upper = 4, 13
    ear_interval, ear_month = early # tuple
    lat_interval, lat_month = late
    interval, month = curr
    interval_rank,month_rank = intervals.index(interval),months.index(month)
    earinterval_rank,earmonth_rank = intervals.index(ear_interval), months.index(ear_month) 
    latinterval_rank,latmonth_rank = intervals.index(lat_interval) or interval_upper, months.index(lat_month) or month_upper
    # check early month rank fits requirement
    if month_rank < earmonth_rank or (month_rank == earmonth_rank and interval_rank < earinterval_rank):
        if interval_rank != 0:
            return False
    # check late month rank fits requirement
    if month_rank > latmonth_rank or (month_rank == latmonth_rank and interval_rank > latinterval_rank):
        return False
    return True







In [86]:
# assumption: 
# 1. move in Month of a house cannot be anytime
# 2. early <= late is valid
def compareMonth(early,late,curr):
    ear_interval, ear_month = early # tuple
    lat_interval, lat_month = late
    interval, month = curr
    interval_rank,month_rank = intervals.index(interval),months.index(month)
    earinterval_rank,earmonth_rank = intervals.index(ear_interval), months.index(ear_month) 
    latinterval_rank,latmonth_rank = intervals.index(lat_interval), months.index(lat_month) or 13
    # check early month rank fits requirement
    early_check = (month_rank > earmonth_rank or (month_rank == earmonth_rank and (interval_rank >= earinterval_rank or interval_rank == 0)))
    # check late month rank fits requirement
    late_check = (month_rank < latmonth_rank or (month_rank == latmonth_rank and (interval_rank <= latinterval_rank or latinterval_rank == 0)))
    return early_check and late_check





In [87]:
test_case1 = (("Anytime","Anytime"),("Anytime","Anytime"),("Anytime","December"))
print(compareMonth(*test_case1) == True)
test_case2 = (("Anytime","January"),("Anytime","March"),("Anytime","February"))
print(compareMonth(*test_case2) == True)
test_case3 = (("Anytime","January"),("Anytime","February"),("Anytime","February"))
print(compareMonth(*test_case3) == True)
test_case4 = (("Anytime","January"),("Anytime","February"),("Anytime","February"))
print(compareMonth(*test_case4) == True)
test_case5 = (("Anytime","January"),("Anytime","February"),("Anytime","January"))
print(compareMonth(*test_case5) == True)
test_case6 = (("Anytime","May"),("Early","November"),("Anytime","November"))
print(compareMonth(*test_case6) == True)
test_case7 = (("Anytime","May"),("Early","November"),("Late","November"))
print(compareMonth(*test_case7) == False)
test_case8 = (("Anytime","May"),("Early","November"),("Anytime","May"))
print(compareMonth(*test_case8) == True)
test_case8 = (("Mid","May"),("Early","November"),("Anytime","May"))
print(compareMonth(*test_case8) == True)
test_case9 = (("Mid","May"),("Early","November"),("Early","May"))
print(compareMonth(*test_case9) == False)
test_case10 = (("Anytime","Anytime"),("Anytime","Anytime"),("Early","December"))
print(compareMonth(*test_case10) == True)
test_case11 = (("Anytime","December"),("Anytime","December"),("Early","December"))
print(compareMonth(*test_case11) == True)
test_case12 = (("Early","December"),("Early","December"),("Early","December"))
print(compareMonth(*test_case12) == True)
test_case12 = (("Mid","December"),("Late","December"),("Early","December"))
print(compareMonth(*test_case12) == False)

True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [78]:
def checkOther(house,request):
    return len(set(request) - set(house)) == 0 

In [81]:
test_case1 = (["Female Only","Male Only", "LGBTQ"],["Female Only","Male Only"])
print(checkOther(*test_case1))
test_case2 = (["Female Only","Male Only", "LGBTQ"],["Female Only","Male Only","420"])
print(checkOther(*test_case2))
test_case3 = (["Female Only","Male Only", "LGBTQ"],["Female Only","Male Only","LGBTQ"])
print(checkOther(*test_case3))
test_case3 = (["Female Only"],["Female Only","Male Only","LGBTQ"])
print(checkOther(*test_case3))

True
False
True
False


In [104]:
import random
def generateOthers():
    num_sample = random.randint(0,7)
    return random.sample(others,num_sample)

In [108]:
def strip(str1):
    return str1[1:]
def replacer(str1):
    return str1.replace(',', '')
import pandas as pd
house_df = pd.read_csv("./housing_example.csv")
house_df['Cost'] = house_df['Cost'].apply(replacer)
house_df['Cost'] = house_df['Cost'].apply(strip)
house_df['Cost'] = house_df['Cost'].apply(int)
# new data frame with split value columns 
new = house_df["Move-in"].str.split(" ", n = 1, expand = True) 
# making separate Interval column from new data frame 
house_df["Interval"]= new[0] 
# making separate Month column from new data frame 
house_df["Month"]= new[1] 
house_df["Others"]= [generateOthers() for _ in range(13)] 

user = [(600,1200), 5.33, 'Single', ('Mid','September'), ('Late','September'),'One Year Lease',['Male Only','Parking','LGBTQ']]

def Filter(price_range, distance, room_type, early, late, duration, other, house):
    
    price_check = price_range[0] <= house['Cost'] and house['Cost'] <= price_range[1]
    dist_check  = house['Distance from Campus'] < distance
    room_check = room_type in house['Room Type']
    move_check = compareMonth(early,late,(house['Interval'],house['Month']))
    dur_check = house['Duration'] == duration
    other_check = checkOther(house['Others'],other)
    
    return price_check and dist_check and room_check and move_check and dur_check and other_check

def run_filter(df, user):
    matches = []
    for i in range(len(df)):
        if Filter(*user, df.iloc[i]):
            matches.append(i)
    return df.iloc[matches].sort_values(['Cost','Distance from Campus'])

run_filter(house_df, user)

,Cost,Duration,Move-in,Distance from Campus,Address,Room Type,Interval,Month,Others
0,700,One Year Lease,Mid September,2.4,Costa Verde Village- 2B2B-,Single/Double,Mid,September,"[Female Only, Parking, LGBTQ, Male Only]"
3,850,One Year Lease,Mid September,2.9,filler,Single,Mid,September,"[Parking, Female Only, Male Only, Pets, LGBTQ,..."
8,1100,One Year Lease,Mid September,2.3,filler,Single,Mid,September,"[LGBTQ, Pets, Male Only, 420, Parking, Female ..."


In [109]:
house_df

,Cost,Duration,Move-in,Distance from Campus,Address,Room Type,Interval,Month,Others
0,700,One Year Lease,Mid September,2.4,Costa Verde Village- 2B2B-,Single/Double,Mid,September,"[Female Only, Parking, LGBTQ, Male Only]"
1,750,One Year Lease,Early September,5.3,e,Single/Double,Early,September,"[420, Parking, Private, Male Only, Pets]"
2,800,One Year Lease,Late September,3.2,filler,Single,Late,September,[LGBTQ]
3,850,One Year Lease,Mid September,2.9,filler,Single,Mid,September,"[Parking, Female Only, Male Only, Pets, LGBTQ,..."
4,900,One Year Lease,Late September,5.1,filler,Single,Late,September,[]
5,950,One Year Lease,Mid September,4.2,filler,Single/Double,Mid,September,"[Private, Female Only, 420, Male Only, LGBTQ]"
6,1000,One Year Lease,Early September,1.0,filler,Single/Double,Early,September,"[Pets, Parking, Female Only, LGBTQ, 420, Male ..."
7,1050,One Year Lease,Early September,1.4,filler,Single,Early,September,"[Private, 420, Male Only, Pets, LGBTQ, Female ..."
8,1100,One Year Lease,Mid September,2.3,filler,Single,Mid,September,"[LGBTQ, Pets, Male Only, 420, Parking, Female ..."
9,1150,One Year Lease,Late September,1.0,filler,Single,Late,September,[Female Only]
